<a href="https://colab.research.google.com/github/Lutris98/21_1Q/blob/master/1.Titanic/Titanic_trial1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. EDA

In [78]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV #model_selection doesn't need mentioning, only the method does
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score #given metric was accuracy


In [79]:
from google.colab import drive #Korean Army banned me from using os resources so I learned new ways
drive.mount('/content/gdrive')
titanic_df=pd.read_csv('/content/gdrive/MyDrive/Dataset/titanic_train.csv') 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [80]:
titanic_df.head(3) 

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [81]:
print(titanic_df.info()) #objects are strings

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [82]:
titanic_feature=titanic_df.drop(['Survived'], axis=1) #parameter axis is mandatory
titanic_label=titanic_df['Survived']

#2. Data Preprocessing

In [83]:
titanic_feature=titanic_feature.drop(['Name', 'Ticket'], axis=1) #1 #PassengerId will be useless but it is how the result should find people #drop method doesn't allow inplacing(Should check results along the way)
titanic_feature['Age'].fillna(titanic_feature['Age'].mean(), inplace=True) #could be affecting, so average would minimize the loss(for further development, something could be inferred from sibsp&parch)
titanic_feature['Cabin'].fillna('N', inplace=True) #inplacing so that additional object would be unnecesory
titanic_feature['Embarked'].fillna('N',inplace=True) #Wonder features like this would be better off set random...
print(titanic_feature.isnull().sum().sum()) #checking

0


In [84]:
print('Sex값 분포:\n',titanic_feature['Sex'].value_counts()) #focusing on columns
print('Cabin값 분포:\n',titanic_feature['Cabin'].value_counts())
print('Embarked값 분포:\n',titanic_feature['Embarked'].value_counts())

Sex값 분포:
 male      577
female    314
Name: Sex, dtype: int64
Cabin값 분포:
 N              687
B96 B98          4
G6               4
C23 C25 C27      4
D                3
              ... 
B73              1
A32              1
B19              1
A24              1
E36              1
Name: Cabin, Length: 148, dtype: int64
Embarked값 분포:
 S    644
C    168
Q     77
N      2
Name: Embarked, dtype: int64


In [85]:
titanic_feature['Cabin']=titanic_df['Cabin'].str[:1] #Cabin needs insight that only the first character contains information #pd.str is applicable to series
features=['Cabin','Sex','Embarked']
for feature in features:
  titanic_feature[feature]=LabelEncoder().fit_transform(titanic_feature[feature].astype(str)) #series의 dtype변형

#3. Cross validation & Hyperparameter tuning

In [86]:
rf_clf=RandomForestClassifier(random_state=0) #Cross validation need models 
lgbm_clf=LGBMClassifier(random_state=0)
lr_clf=LogisticRegression(max_iter=100000)
X_train, X_test, y_train, y_test=train_test_split(titanic_feature, titanic_label, test_size=0.1,random_state=0)

In [87]:
rf_params={'n_estimators':[300],
           'max_depth':[6,8,10,12], #similar to decisiontree
           'min_samples_leaf':[8,12,18],
           'min_samples_split':[8,16,20]}
rf_gridcv=GridSearchCV(rf_clf, param_grid=rf_params, cv=5, n_jobs=-1) #using all CPU cores
rf_gridcv.fit(X_train, y_train)
print('best parameters:\n',rf_gridcv.best_params_)

best parameters:
 {'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 20, 'n_estimators': 300}


In [88]:
lgbm_params={'n_estimators':[300],
             'num_leaves':[32,64],
             'min_child_samples':[60,100],
             'subsample':[0.8,1],
             'max_depth':[160]}
lgbm_gridcv=GridSearchCV(lgbm_clf, param_grid=lgbm_params, cv=5, n_jobs=-1) 
lgbm_gridcv.fit(X_train, y_train)
print('best parameters:\n',lgbm_gridcv.best_params_)

best parameters:
 {'max_depth': 160, 'min_child_samples': 100, 'n_estimators': 300, 'num_leaves': 32, 'subsample': 0.8}


In [89]:
lr_params={'penalty':['l2','elasticnet'],'C':[0.1,1,5,10]}
lr_gridcv=GridSearchCV(lr_clf, param_grid=lr_params, cv=5, n_jobs=-1) 
lr_gridcv.fit(X_train, y_train)
print('best parameters:\n',lr_gridcv.best_params_)

best parameters:
 {'C': 0.1, 'penalty': 'l2'}


#4. modeling

In [91]:
rf_clf=RandomForestClassifier(n_estimators=300, max_depth=6, min_samples_leaf=8, min_samples_split=20, random_state=0) #Cross validation need models 
lgbm_clf=LGBMClassifier(n_estimators=300, num_leaves=32, min_child_samples=100, subsample=0.8, max_depth=160, random_state=0)
lr_clf=LogisticRegression(penalty='l2',C=0.1)
vo_clf=VotingClassifier(estimators=[('RandomForest',rf_clf),('LightGBM',lgbm_clf), ('LogisticRegression',lr_clf)], voting='soft')
vo_clf.fit(X_train, y_train)
pred=vo_clf.predict(X_test)
print('Final accuracy:{0:.4f}'.format(accuracy_score(y_test,pred))) #formatting by 0:

Final accuracy:0.8444


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


내가 만든 첫번째 인공지능!<br>
타이타닉에서 살아남은 사람을 84%밖에 예측하지 못했다ㅋㅋㅋ<br>
90%이상으로 끌어올려보자!<br>